In [20]:
import os
import re
import glob
import pickle
import numpy as np

In [21]:
root_dir = os.path.abspath('../')
data_dir = os.path.join(root_dir, 'data')
res_dir = os.path.join(root_dir, 'res')
res_filtered_dir = os.path.join(res_dir, 'filtered')
res_raw_dir = os.path.join(res_dir, 'raw')
res_raw_vocab = os.path.join(res_raw_dir, 'vocab')
res_raw_wti = os.path.join(res_raw_dir, 'wti')
res_raw_embeddings = os.path.join(res_raw_dir, 'embed')
res_filt_vocab = os.path.join(res_filtered_dir, 'vocab')
res_filt_wti = os.path.join(res_filtered_dir, 'wti')
res_filt_embeddings = os.path.join(res_filtered_dir, 'embed')
raw_data_dir = os.path.join(data_dir,'raw_data')
clean_data_dir = os.path.join(data_dir, 'clean_data')
embeddings_dir = os.path.join(data_dir, 'glove.twitter.27B')
embeddings_file = os.path.join(embeddings_dir, 'glove.twitter.27B.200d.txt')
print embeddings_file
print embeddings_dir

/home/vasanth/programming/690N/project/conversational_agent_personified/data/glove.twitter.27B/glove.twitter.27B.200d.txt
/home/vasanth/programming/690N/project/conversational_agent_personified/data/glove.twitter.27B


In [22]:
from nltk.tokenize import wordpunct_tokenize
from nltk.stem.porter import *

stemmer = PorterStemmer()

def stem_if_possible(word):
    word = word.lower()
    word = stemmer.stem(word)
    return word
        
def construct_vocabulary():
    words = []
    res_word_to_index = []
    res_embeddings = []
    print "Extract vocabulary..."
    files_processed = 0
    for fname in glob.iglob(res_dir + '/*_ascii.txt'):
        print fname
        files_processed += 1
        for line in open(fname):
            words += [stem_if_possible(w) for w in list(set(wordpunct_tokenize(line)))]
    print "Files processed {0}".format(files_processed)
    print len(list(set(words)))
    print "Now read from twitter embeddings and update"
    word_list = list(set(words))
    for word in word_list:
        for fname in glob.iglob(res_raw_wti + '/*.pickle'):
            vocab_partial = pickle.load(open(fname,"rb"))
            vocab_partial = [stem_if_possible(v) for v in vocab_partial]
            if word in vocab_partial:
                file_index = int(filter(str.isdigit, fname.split("/")[-1]))
                embed_file = "embedding"+str(file_index)+".npy"
                embed_file_full = os.path.join(res_raw_embeddings,embed_file)
                embed_partial = np.load(embed_file_full)
                print "Word: {0} index {1}".format(word, file_index)
                res_word_to_index.append(word)
                res_embeddings.append(embed_partial[vocab_partial[word]])
    print "Done with vocab extraction"
    return word_list

import re
import string
printable = set(string.printable)
func = lambda s: filter(lambda x: x in printable, s)

def write_our_words():
    words = []
    fnames = [res_dir + '/input1.txt', res_dir + '/input2.txt']
    for fname in fnames:
        print fname
        for line in open(fname):
#             line = re.sub(r'(\.)+', '.', line)
#             line = re.sub(r'(\?)+', '?', line)
#             line = re.sub(r'(\')+', '\'', line)
#             line = re.sub(r'(\!)+', '!', line)
#             line = re.sub('(?<! )(?=[.,!?()])|(?<=[.,!?()])(?! )', r'', line)
#             words += [func(w) for w in line.lower().split()]
            line = line.translate(None, string.punctuation)
            print line
            words += [func(w) for w in line.lower().split()]
    with open('vocab.txt', 'w') as f:
        f.write('\n'.join(list(set(words))))

In [17]:
write_our_words()

/home/vasanth/programming/690N/project/conversational_agent_personified/res/input1.txt
/home/vasanth/programming/690N/project/conversational_agent_personified/res/input2.txt


In [14]:
cv = construct_vocabulary()

Extract vocabulary...
/home/vasanth/programming/690N/project/conversational_agent_personified/res/input3_ascii.txt


UnicodeDecodeError: 'ascii' codec can't decode byte 0xe0 in position 2: ordinal not in range(128)

In [5]:
def question_answer_format():
    vocabulary = {}
    if not os.path.exists(res_raw_wti):
        os.makedirs(res_raw_wti)
    if not os.path.exists(res_raw_wti):
        os.makedirs(res_raw_wti)
    questions_file = os.path.join(res_dir,'input1.txt')
    answers_file = os.path.join(res_dir,'input2.txt')
    questions_file1 = os.path.join(res_dir,'input3.txt')
    answers_file1 = os.path.join(res_dir,'input4.txt')    
    print "Start writing ques answer files..."
    import glob
    lines_written = 0
    files_processed = 0
    for full_filename in glob.iglob(clean_data_dir + '/*/*.txt'):
        files_processed += 1
        with open(full_filename) as f1:
            lines = f1.readlines()
            with open(questions_file, 'a') as f2, open(answers_file, 'a') as f3, open(questions_file1, 'a') as f4, open(answers_file1, 'a') as f5:                            

                for i, line in enumerate(lines):
                        line = line.split(":")[1]
                        line1 = re.sub(r'\([^)]*\)', '', line).strip(" ")
                        f2.write(line)
                        f4.write(line1)
                        if lines_written == 0:
                            pass          
                        elif lines_written % 100000 == 0:
                            f3.write(line)
                            f5.write(line1)
                            print "Lines written: {0}".format(lines_written)
                            print "Writing from {0}".format(full_filename.split("/")[-1])
                        else:
                            f3.write(line)
                            f5.write(line1)
                        lines_written += 1
    print "Finished writing all files."
    print "{0} files processed. {1} lines written".format(files_processed, lines_written)

In [6]:
question_answer_format()

Start writing ques answer files...
Lines written: 100000
Writing from Ordinary People.txt
Lines written: 200000
Writing from My Mother Dreams the Satan's Disciples in New York.txt
Lines written: 300000
Writing from State and Main.txt
Lines written: 400000
Writing from Garden State.txt
Lines written: 500000
Writing from Precious.txt
Lines written: 600000
Writing from Being There.txt
Lines written: 700000
Writing from Analyze That.txt
Lines written: 800000
Writing from Two For The Money.txt
Finished writing all files.
1052 files processed. 839983 lines written


In [3]:
def loadWordEmbeddings(filename):
    #vocabulary = []
    word_to_index = {}
    embeddings = []
    i = 0
    file_point = 0
    if not os.path.exists(res_raw_embeddings):
        os.makedirs(res_raw_embeddings)
    if not os.path.exists(res_raw_wti):
        os.makedirs(res_raw_wti)
    for line in open(filename,'r'):
        one_record = line.strip().split(" ")
        #vocabulary.append(one_record[0])
        word_to_index[one_record[0]] = i%500000
        embeddings.append(one_record[1:])
        i += 1
        if i%500000 == 0:
            print "Embedding of {0} words done. Current token: {1}, isCapital : {2}".format(i, one_record[0], one_record[0].islower())
            print "Partial reading twitter glove vocab done. |V| = {0}, |embed| = {1}".format(len(word_to_index),len(embeddings))
            
            print "Now pickling word_to_index dict..."
            wti = "wti"+str(file_point)+".pickle"
            embed = "embedding"+str(file_point)+".npy"
            wti_filename = os.path.join(res_raw_wti, wti)
            embed_filename = os.path.join(res_raw_embeddings, embed)
            with open(wti_filename, 'wb') as handle:
                pickle.dump(word_to_index, handle, protocol=pickle.HIGHEST_PROTOCOL)
            print "Pickling word_to_index dict complete."
            print "Now pickling embeddings array..."
            np.save(embed_filename, embeddings, allow_pickle=True)
            print "Pickling embeddings array complete."                
            print "Loaded embeddings!!"            
            word_to_index = {}
            embeddings = []
            file_point += 1
    print "Embedding of {0} words done. Current token: {1}, isCapital : {2}".format(i, one_record[0], one_record[0].islower())
    print "Partial reading twitter glove vocab done. |V| = {0}, |embed| = {1}".format(len(word_to_index),len(embeddings))

    print "Now pickling word_to_index dict..."
    wti = "wti"+str(file_point)+".pickle"
    embed = "embedding"+str(file_point)+".npy"
    wti_filename = os.path.join(res_raw_wti, wti)
    embed_filename = os.path.join(res_raw_embeddings, embed)
    with open(wti_filename, 'wb') as handle:
        pickle.dump(word_to_index, handle, protocol=pickle.HIGHEST_PROTOCOL)
    print "Pickling word_to_index dict complete."
    print "Now pickling embeddings array..."
    np.save(embed_filename, embeddings, allow_pickle=True)
    print "Pickling embeddings array complete."                
    print "Loaded embeddings!!"            
    print "Finished reading twitter glove vocab"
    #return word_to_index, embeddings
    
            

In [4]:
embd_file_dir = os.path.join(data_dir, 'glove.twitter.27B')
embd_file_name = os.path.join(embd_file_dir,'glove.twitter.27B.200d.txt')
loadWordEmbeddings(embd_file_name)
#print word_to_index
#print embeddings

Embedding of 500000 words done. Current token: تغيّروُآ, isCapital : False
Partial reading twitter glove vocab done. |V| = 500000, |embed| = 500000
Now pickling word_to_index dict...
Pickling word_to_index dict complete.
Now pickling embeddings array...
Pickling embeddings array complete.
Loaded embeddings!!
Embedding of 1000000 words done. Current token: ikatannya, isCapital : True
Partial reading twitter glove vocab done. |V| = 500000, |embed| = 500000
Now pickling word_to_index dict...
Pickling word_to_index dict complete.
Now pickling embeddings array...
Pickling embeddings array complete.
Loaded embeddings!!
Embedding of 1193517 words done. Current token: <unk>, isCapital : True
Partial reading twitter glove vocab done. |V| = 193515, |embed| = 193517
Now pickling word_to_index dict...
Pickling word_to_index dict complete.
Now pickling embeddings array...
Pickling embeddings array complete.
Loaded embeddings!!
Finished reading twitter glove vocab


Start filtering word embeddings...


0.80444

['fawn',
 'Sergeantthis',
 'MargaretI',
 'HURTLE',
 'Pecrowsky',
 'mussss',
 'nunnery',
 'EXPLAIN',
 'Dago',
 'INTRUDERS',
 'Pront',
 'Popscile',
 'Plasticine',
 'teleflorists',
 'woods',
 'clotted',
 'spiders',
 'hanging',
 'woody',
 'somepla',
 'Jaundice',
 'comically',
 'localized',
 'spidery',
 'crasait',
 'disobeying',
 'caner',
 'canes',
 'LAST',
 'ition',
 'ChristDecember',
 'chatter',
 'OPAL',
 'Craziness',
 'sociopathy',
 'e_',
 'Mmrrunm',
 'lency',
 'DISHEARTENED',
 'Vengeance',
 'caned',
 'Western',
 'Writerscards',
 'crossbar',
 'LASH',
 '_whole_',
 'isMy',
 'Retreat',
 'Euro',
 'Tootie',
 'Debts',
 'Mortes',
 'Waziri',
 'savaient',
 'WINNIE',
 'Valle',
 'Famed',
 'Blade',
 'itive',
 'GALLARDO',
 'pigment',
 'kid\xc2',
 'kid\xc3',
 'WATERMELON',
 'Guvner',
 'rying',
 'PORTLAND',
 'Booteria',
 'screaming',
 'FAVs',
 'prizing',
 'wooded',
 'inseminoid',
 'Reconcile',
 'Baaaaaaa',
 'wooden',
 'SULTAN',
 'Sergeants',
 'Sack',
 'viable',
 'circuitry',
 'crotch',
 'WINCE',
 'WINC

In [ ]:
# Filter the words based on the unigram count in the files
# Create the vocab file
# Filter the embeddings to only include the words in vocab
# Create fake embedding for _PAD, UNK, _GO, _EOS add them at start of vocab